In [87]:
import unittest
import pandas as pd
import mechanize
from Bio.Data.IUPACData import protein_letters_3to1
import mutfunc_functionality

In [79]:
runmutfunc("Hickman2017mut.xlsx")

In [89]:
df = pd.read_excel("test/test_mutation_file.xlsx", header=4, keep_default_na=False)
mutfunc_functionality.extract_snps(df).to_csv("test/test_extract_snps.csv", index=False)